<a href="https://colab.research.google.com/github/Srinidhireddy25/AIML-2025/blob/main/lab_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>